# Working with AGOL items - *Streamlined*

---
Fall 2024  
John.Fay@duke.edu

---

In [1]:
#Import basic packages
import pandas as pd

#Import the arcgis "GIS" object and connect to AGOL
from arcgis import GIS
gis = GIS()

#Import other classes
from arcgis.features import FeatureLayer, GeoAccessor
from arcgis.geocoding import geocode
from arcgis.geometry import Point, filters

In [2]:
flood_item = gis.content.get('2b245b7f816044d7a779a61a5844be23')
flood_item

<Item title:"USA Flood Hazard Areas" type:Feature Layer Collection owner:esri_landscape2>

In [3]:
#Get its layer
flood_layer = flood_item.layers[0]
flood_layer.url

'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Flood_Hazard_Reduced_Set_gdb/FeatureServer/0'

In [5]:
#Filter for dfirmID
flood_fset = flood_layer.query(where="DFIRM_ID LIKE '37063%'")
flood_fset

<FeatureSet> 17158 features

In [ ]:
flood_fset.sdf.spatial.plot()

In [ ]:
#Find its spatial reference from the item
flood_sr = flood_item.spatialReference
flood_sr

In [ ]:
#Define an area to subset the data - we'll use Durham
durham_results = geocode("Durham City, NC")
durham_coords = durham_results[0]['location']
durham_coords['spatialReference'] = {'wkid':4326}
durham_point = Point(durham_coords)
durham_point_prj = durham_point.project_as(flood_sr)
durham_filter = filters.intersects(durham_point_prj.buffer(1000))
durham_filter

In [ ]:
#Apply the filter
durham_flood_fset = flood_layer.query(
    geometry_filter=durham_point
)
type(durham_flood_fset)

In [ ]:
print(durham_point)

In [ ]:
m = gis.map('Durham, NC')
m.draw(durham_point_prj)
m.draw(durham_point_prj.buffer(1))
m